In [ ]:
# set up Dependencies
import pandas as pd
import numpy as np
import requests
import time
import os
import matplotlib.pyplot as plt
from scipy.stats import linregress
from citipy import citipy

# import api key from environment variables
weather_api_key = os.environ.get('WEATHER_API_KEY')

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set(set_size) of random lat and lng combinations
set_size=250
lats = np.random.uniform(lat_range[0], lat_range[1], size=set_size)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=set_size)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1])
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(len(cities))